In [5]:
import pandas as pd 

df = pd.read_pickle("/workspace/LASR-probe-gen/datasets/test/meta-llama_Llama-3.2-3B-Instruct/__off_policy_prompt/outputs__off_policy_prompt_activations.pkl")

df

,input_formatted,input,model_outputs,activations
0,<|begin_of_text|><|start_header_id|>system<|en...,How can I get free internet from my neighbor's...,system\n\nCutting Knowledge Date: December 202...,"{18: [[tensor(-0.2964, dtype=torch.float16), t..."
1,<|begin_of_text|><|start_header_id|>system<|en...,Can you tell me about Lasik eye surgery? I am ...,system\n\nCutting Knowledge Date: December 202...,"{18: [[tensor(-0.2959, dtype=torch.float16), t..."


In [2]:
import json
from probe_gen.paths import data

file_path = data.metaphors_brazil / "llama_3b_brazil_prompted_balanced_1k.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        data = json.loads(line)
        print(f"\nRow {i}:")
        for key, value in data.items():
            preview = value
            if isinstance(value, str) and len(value) > 200:
                preview = value[:200] + "..."
            print(f"  {key}: {preview}")


llama_3b_ultrachat_brazilian.jsonl:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

Downloaded to: /workspace/LASR-probe-gen/data/ultrachat_brazil/llama_3b_ultrachat_brazilian.jsonl


qwen_3b_outputs.jsonl:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

Downloaded to: /workspace/LASR-probe-gen/data/ultrachat_brazil/qwen_3b_outputs.jsonl


In [ ]:
from openai import OpenAI

import os

client = OpenAI(api_key="")

def translate(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-4" if available
        messages=[
            {"role": "user", "content": f"Translate the following from Portuguese to English:\n\n{text}"}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


In [5]:
import json
import random
import pandas as pd
from probe_gen.paths import data

# Load all data
file_path = data.metaphors_brazil / "qwen_3b_brazil_prompted_balanced_1k.jsonl"
with open(file_path, "r", encoding="utf-8") as f:
    lines = [json.loads(line) for line in f]

# Sample 10 random rows
sampled = random.sample(lines, 10)

# Create a clean list of dictionaries for export
rows = []
for i, ex in enumerate(sampled):
    row = {
        "Portuguese Input": ex.get("input", ""),
        "English Input": translate(ex.get("input", "")),
        "Model Output (PT)": ex.get("model_outputs", ""),
        "Model Output (EN)": translate(ex.get("model_outputs", "")),
        "Labels": ex.get("labels", ""),
        "Scale Label": ex.get("scale_labels", ""),
        "Explanation (PT)": ex.get("scale_label_explanation", ""),
        "Explanation (EN)": translate(ex.get("scale_label_explanation", "")),
    }
    rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

# Save to Excel or PDF
csv_path = data.metaphors_brazil / "sampled_translations_qwen-3b-brazil-prompted-balanced-1k.csv"
df.to_csv(csv_path, index=False)
print(f"Saved to {csv_path}")


Saved to /workspace/LASR-probe-gen/data/metaphors_brazil/sampled_translations_qwen-3b-brazil-prompted-balanced-1k.csv
